In [ ]:
import pandas as pd
from google.colab import files
from google.colab import drive
import datetime
from tqdm import tnrange, tqdm_notebook, tqdm

In [ ]:
#Progress bar pandas
tqdm.pandas()

#Loading in data

In [ ]:
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Knab/Data/Knab_Analytics/"

In [ ]:
df_urls = pd.read_excel("/content/drive/My Drive/Knab/Data/Articles/URLS.xlsx")
df_data = pd.read_csv("/content/drive/My Drive/Knab/Data/Knab_Analytics/merged_data.csv")

In [ ]:
df_data

# Find old recommendations

In [ ]:
def find_old_recommendations(url,visit_time): #URL in string 'url' and visit_time in date_time
  if url in df_urls['URL'].values:
    tag = df_urls.loc[df_urls['URL'].isin([url])]['TAG'].iloc[0] #Find tag or current article
    tag_list = df_urls.loc[df_urls['TAG'].isin([tag])] #Find the list of articles with the same tag
    tag_list['DATE'] = pd.to_datetime(tag_list['DATE']) - pd.to_datetime(visit_time)

    older_articles = tag_list[(tag_list['DATE'] < pd.Timedelta(0))] #Remove all positive time difference, we only want older articles
    old_recommendations = older_articles.nlargest(3,'DATE')['URL'].values[:] #Find 3 most recent recommendations for the visit_time
  else:
    old_recommendations = 0

  return old_recommendations

In [ ]:
df_articles = df_data[df_data['ArticleYN'] == 1]

In [ ]:
#Initialize column
df_data['old_recommendations'] = df_data[['URL','visitstarttime']].progress_apply(lambda x: find_old_recommendations(*x), axis=1)